In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from skimage.metrics import structural_similarity
import pandas as pd
from IPython.display import display, HTML
import base64
import logging

# Configuration
INPUT_ROOT = 'DATASET'
OUTPUT_ROOT = 'DATASET_bilateral'
SSIM_THRESHOLD = 0.85
MAX_PREVIEW = 20

# Parameters for bilateral filter (d, sigmaColor, sigmaSpace)
PARAM_COMBINATIONS = [
    (5, 50, 50),
    (7, 75, 75),
    (9, 100, 100),
    (11, 125, 125),
    (13, 150, 150)
]

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s', handlers=[logging.StreamHandler()])

def calculate_ssim(original, denoised):
    """Calculate SSIM between the original and denoised images."""
    return structural_similarity(original, denoised, data_range=255)

def find_optimal_bilateral_params(image):
    """Find the optimal Bilateral filter parameters based on SSIM."""
    gray_orig = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    best_params = (5, 50, 50)
    best_ssim = 0

    for d, sigmaColor, sigmaSpace in PARAM_COMBINATIONS:
        denoised = cv2.bilateralFilter(image, d, sigmaColor, sigmaSpace)
        gray_denoised = cv2.cvtColor(denoised, cv2.COLOR_BGR2GRAY)
        ssim_val = calculate_ssim(gray_orig, gray_denoised)

        if ssim_val >= SSIM_THRESHOLD:
            best_params = (d, sigmaColor, sigmaSpace)
            best_ssim = ssim_val
        else:
            break  # Stop early if SSIM drops
    return best_params, best_ssim

def encode_image_for_html(img):
    """Encode image into base64 for embedding into HTML."""
    _, buffer = cv2.imencode('.jpg', img)
    b64 = base64.b64encode(buffer).decode()
    return f'<img src="data:image/jpeg;base64,{b64}" width="100"/>'

def process_image(img_path):
    """Process a single image and return results."""
    try:
        image = cv2.imread(img_path)
        if image is None:
            raise ValueError(f"Image could not be read: {img_path}")

        # Find optimal bilateral filter params
        params, final_ssim = find_optimal_bilateral_params(image)
        d, sigmaColor, sigmaSpace = params
        denoised = cv2.bilateralFilter(image, d, sigmaColor, sigmaSpace)

        # Save the denoised image
        split, class_name = img_path.split(os.sep)[-3:-1]
        output_path = os.path.join(OUTPUT_ROOT, split, class_name, os.path.basename(img_path))
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        cv2.imwrite(output_path, denoised)

        return img_path, params, final_ssim, denoised
    except Exception as e:
        logging.error(f"Error processing {img_path}: {str(e)}")
        return img_path, None, None, None

def process_dataset_bilateral():
    """Process the dataset with Bilateral filtering and compute SSIM."""
    report = []
    total_ssim = 0
    processed_count = 0

    # Calculate total images for progress bar
    total_images = sum(
        len(files)
        for _, _, files in os.walk(INPUT_ROOT)
        if any(f.lower().endswith(('.png', '.jpg', '.jpeg')) for f in files)
    )

    # Initialize progress bar
    progress = tqdm(total=total_images, desc="🎯 Bilateral Filtering", unit="img")

    # Process images
    for split in os.listdir(INPUT_ROOT):
        split_path = os.path.join(INPUT_ROOT, split)
        if not os.path.isdir(split_path):
            continue

        for class_name in os.listdir(split_path):
            class_path = os.path.join(split_path, class_name)
            if not os.path.isdir(class_path):
                continue

            for filename in os.listdir(class_path):
                if not filename.lower().endswith(('.png', '.jpg', '.jpeg')):  # Skip non-image files
                    logging.warning(f"Skipping non-image file: {filename}")
                    continue

                img_path = os.path.join(class_path, filename)

                # Process image
                img_path, params, final_ssim, denoised = process_image(img_path)

                if params is None:  # Skip if image processing failed
                    continue

                # Store preview image for table
                preview = encode_image_for_html(denoised) if processed_count < MAX_PREVIEW else ""

                # Update report with details
                report.append({
                    'Image': preview,
                    'Path': img_path,
                    'Params': f"(d={params[0]}, σC={params[1]}, σS={params[2]})",
                    'SSIM': f"{final_ssim:.4f}"
                })

                total_ssim += final_ssim
                processed_count += 1
                progress.update(1)

    progress.close()
    avg_ssim = total_ssim / processed_count if processed_count > 0 else 0
    return report, avg_ssim

def show_html_table(report, avg_ssim):
    """Display a table in HTML format with image previews, parameters, and SSIM."""
    df = pd.DataFrame(report[:MAX_PREVIEW])
    logging.info(f"\n✅ Average SSIM: {avg_ssim:.2%} across {len(report)} images.")
    display(HTML(df.to_html(escape=False, index=False)))

if __name__ == "__main__":
    report, avg_ssim = process_dataset_bilateral()
    show_html_table(report, avg_ssim)


🎯 Bilateral Filtering:   0%|          | 12/2853 [00:11<45:27,  1.04img/s]

: 